## 대용량 데이터(5G) 용량 줄이기
1. csv 파일 로드하기 : pd.read_csv('data/2019-Oct.csv')
2. column 별 데이터 값 확인하여 필요 없는 col 없애기 : df.drop(columns=[])
3. 각 열 데이터 형식 확인하기
4. 각 열 downcast 하기 : pd.to_numeric(df[col], downcast='')
 => for 문을 통해 자동으로 변환하기
5. parquet.gzip으로 파일 저장하기 : df.to_parquet(file_name, index=False)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/2019-Oct.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 2.8+ GB


In [3]:
df_down = df.copy()
df_down.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 2.8+ GB


In [4]:
df['event_type'].unique()

array(['view', 'purchase', 'cart'], dtype=object)

In [5]:
# data type 이 object인 열이 범주형 데이터가 들어있는지 확인하기
# event_time 은 날짜 데이터,  
for col in df_down.columns:
    if df_down[col].dtypes.name.startswith('object'):
        print(f'{col} nunique : {df[col].nunique()}')

event_time nunique : 2621538
event_type nunique : 3
category_code nunique : 126
brand nunique : 3444
user_session nunique : 9244421


In [8]:
df_down.isnull().sum()

event_time              0
event_type              0
product_id              0
category_id             0
category_code    13515609
brand             6117080
price                   0
user_id                 0
user_session            2
dtype: int64

In [7]:
for col in df_down.columns:
    
    if df_down[col].dtypes.name.startswith('int'):
        if df_down[col].min() < 0:
            df_down[col] = pd.to_numeric(df_down[col], downcast='integer')
        else:
            df_down[col] = pd.to_numeric(df_down[col], downcast='unsigned')
    
    elif df_down[col].dtypes.name.startswith('float'):
        df_down[col] = pd.to_numeric(df_down[col], downcast='float')
        
    elif df_down[col].dtypes.name.startswith('object') & (df_down[col].nunique() < 3500):
        df_down[col] = df_down[col].astype('category') 

In [9]:
df_down.describe()

,product_id,category_id,price,user_id
count,4.244876e+07,4.244876e+07,4.244876e+07,4.244876e+07
mean,1.054993e+07,2.057404e+18,2.903238e+02,5.335371e+08
std,1.188191e+07,1.843926e+16,3.363253e+02,1.852374e+07
min,1.000978e+06,2.053014e+18,0.000000e+00,3.386938e+07
25%,1.005157e+06,2.053014e+18,6.598000e+01,5.159043e+08
50%,5.000470e+06,2.053014e+18,1.629300e+02,5.296965e+08
75%,1.600030e+07,2.053014e+18,3.585700e+02,5.515788e+08
max,6.050001e+07,2.175420e+18,2.574070e+03,5.662809e+08


In [10]:
df_down.isnull().sum()

event_time              0
event_type              0
product_id              0
category_id             0
category_code    13515609
brand             6117080
price                   0
user_id                 0
user_session            2
dtype: int64

In [11]:
df_down = df_down.drop(columns=['category_code', 'user_session'])
df_down.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 7 columns):
 #   Column       Dtype   
---  ------       -----   
 0   event_time   object  
 1   event_type   category
 2   product_id   uint32  
 3   category_id  uint64  
 4   brand        category
 5   price        float32 
 6   user_id      uint32  
dtypes: category(2), float32(1), object(1), uint32(2), uint64(1)
memory usage: 1.2+ GB


In [12]:
df_down['event_time'] = pd.to_datetime(df_down['event_time'])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 2.8+ GB


In [14]:
df_down.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 7 columns):
 #   Column       Dtype              
---  ------       -----              
 0   event_time   datetime64[ns, UTC]
 1   event_type   category           
 2   product_id   uint32             
 3   category_id  uint64             
 4   brand        category           
 5   price        float32            
 6   user_id      uint32             
dtypes: category(2), datetime64[ns, UTC](1), float32(1), uint32(2), uint64(1)
memory usage: 1.2 GB


# parquet 포멧으로 파일 저장하기
- donwcast 후 csv 파일과 parquet 저장한 파일 용량 비교하기

In [15]:
import os

In [16]:
def convert_bytes(num):
    """
    1024 보다 크면 숫자를 나누고 아니면 숫자와 단위를 표시하도록
    for문을 돌면서 값을 1024로 나누고 
    값이 1024 보다 작다면 단위와 함께 num 을 반횐
    """
    for file_size in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024:
            return f'{num:.2f}{file_size}'
            break
        num /= 1024

In [17]:
def file_size(file_path):
    """
    파일이 있다면 convert_bytes 함수를 통해 크기를 구함
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)

In [18]:
def compare_csv_parquet(df):
    """
    데이터프레임을 csv 와 parquet형식으로 저장하하고 각 파일 사이즈를 dict 형태로 반환
    """
    file_path_parquet = 'data/2019-Oct_downcast.parquet.gzip'
    file_path_csv = 'data/2019-Oct.csv'
    
    df_down.to_parquet(file_path_parquet, compression='gzip', index=False)
    
    return ({'parquet': file_size(file_path_parquet), 'csv': file_size(file_path_csv)})

In [19]:
compare_csv_parquet(df_down)

{'parquet': '377.87MB', 'csv': '5.28GB'}